## Notebook responsável pela geração do mapa utilizado pelo SVAA

In [ ]:
import base64
import os

import folium
import geemap.foliumap as geemap
import geopandas as gpd
import leafmap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests as req
from folium import plugins
from folium.plugins import FastMarkerCluster, MarkerCluster
from ipyleaflet import (AwesomeIcon, CircleMarker, DivIcon, Icon, LayerGroup,
                        Map, Marker, Popup)
from ipywidgets import HTML
from rgb_gradient import get_linear_gradient
from scipy.stats import spearmanr
from shapely.geometry import LineString, Point, Polygon

### Carrega o Dataset do mapa

In [ ]:
path_dataset_para_mapa = "../../dados/util/df_mapa.xlsx"

df_mapa = pd.read_excel(path_dataset_para_mapa)

### Cria datasets a partir do df_mapa para visualização de diferentes camadas no mapa

In [ ]:
df_mapa_score_0 = df_mapa.query("`Score Normalizado` == 0")

df_mapa_score_entre_0_e_25 = df_mapa.query(
    "`Score Normalizado` > 0 and `Score Normalizado` < 25"
)

df_mapa_score_entre_25_e_50 = df_mapa.query(
    "`Score Normalizado` >= 25 and `Score Normalizado` < 50"
)

df_mapa_score_entre_50_e_75 = df_mapa.query(
    "`Score Normalizado` >= 50 and `Score Normalizado` < 75"
)

df_mapa_score_entre_75_e_100 = df_mapa.query(
    "`Score Normalizado` >= 75 and `Score Normalizado` <= 100"
)

### Definição das cores que serão utilizadas no mapa 

In [ ]:
corAmarela = "#FDBE85"
corBeje = "#FD8D3C"
corLaranja = "#E6550D"
corLaranjaEscuro = "#A63603"

cores_intermediarias_gradiente = [corAmarela, corBeje, corLaranja, corLaranjaEscuro]


def get_color_discrete(score):
    if score is np.nan:
        return "#000000"
    elif score < 25:
        return corAmarela
    elif score < 50:
        return corBeje
    elif score < 75:
        return corLaranja
    else:
        return corLaranjaEscuro

### Carrega atributos necessários para a exibição do mapa

In [ ]:
# calcula onde vai começar centralizado o mapa
media_x = df_mapa["Latitude"].mean()
media_y = df_mapa["Longitude"].mean()

# carrega os polígonos dos Estados do Brasil
url_poly_brasil = "https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF"

headers = {"Accept": "application/vnd.geo+json"}

mapa_brasil = req.get(url_poly_brasil, headers=headers)

poly_brasil = mapa_brasil.json()

### Funções utilizadas pelo mapa

In [ ]:
# Função que adiciona camadas ao mapa


def adiciona_camadas(map_object, data, nome_camada):
    camada = folium.FeatureGroup(name=nome_camada)
    for _, row in data.iterrows():
        color = get_color_discrete(row["Score Normalizado"])
        popup_content = f"""
            <div>
                <div style="font-size: 1.1em;">
                    <b>Subestação:</b> {row['Local de instalação']}<br>
                </div>
                <div style="margin-top: 10px;">
                    <b>Quantidade de OS:</b> {row['qtd de OS']}<br>
                    <b>Quantidade de Acidentes:</b> {row['qtd de acidentes']}<br>
                    <b>Potencial Acidentes:</b> {round(row['peso acidentes'], 2)}<br>
                    <b>Quantidade de acidentes com alto potencial :</b> {round(row['qtd de acidentes alto potencial'])}<br>
                    <b>Métrica de Risco:</b> {round(row['Score'], 2)}
                </div>
            </div>
        """
        folium.Marker(
            location=(row["Latitude"], row["Longitude"]),
            icon=folium.DivIcon(
                html=f"""
                <div style="
                    background-color: {color};
                    color: white;
                    border-radius: 50%;
                    width: 50px;
                    height: 50px;
                    display: flex;
                    justify-content: center;
                    align-items: center;
                    font-size: 14px;
                    font-weight: bold;
                    transform: translate(-50%, -50%);
                ">
                    {round(row['Score Normalizado'], 2)}
                </div>
            """
            ),
            popup=folium.Popup(popup_content, max_width=2650),
            zIndexOffset=round(row["Score Normalizado"] * 100),
        ).add_to(camada)

    map_object.add_child(camada)

### Criação do Mapa

In [ ]:
# Criar o mapa centrado nas coordenadas especificadas
mapa = geemap.Map(
    center=[media_x, media_y],
    zoom=7,
    min_zoom=4,
    search_control=False,
    min_lat=-45,
    max_lat=15,
    min_lon=-85,
    max_lon=-20,
    max_bounds=True,
)

# Adiciona html de legenda ao mapa
caminho_legenda_html = "../../assets/legenda.html"
with open(caminho_legenda_html, "r", encoding="utf-8") as arquivo:
    legenda_html = arquivo.read()

mapa.get_root().html.add_child(folium.Element(legenda_html))

# Adicionar a camada de polígonos de estados do Brasil
camada_poligonos_brasil = folium.FeatureGroup(name="Polígonos do Brasil")
folium.GeoJson(
    poly_brasil,
    zoom_on_click=True,
    style_function=lambda feature: {
        "fillColor": "#c8d977",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
        "fillOpacity": 0.15,
    },
).add_to(camada_poligonos_brasil)
mapa.add_child(camada_poligonos_brasil)

# Adicionar camadas de visualização ao mapa
adiciona_camadas(mapa, df_mapa_score_0, "Exibir probabilidades NULAS")
adiciona_camadas(
    mapa, df_mapa_score_entre_0_e_25, "Exibir probabilidades entre 0 e 25 %"
)
adiciona_camadas(
    mapa, df_mapa_score_entre_25_e_50, "Exibir probabilidades entre 25 e 50 %"
)
adiciona_camadas(
    mapa, df_mapa_score_entre_50_e_75, "Exibir probabilidades entre 50 e 75 %"
)
adiciona_camadas(
    mapa, df_mapa_score_entre_75_e_100, "Exibir probabilidades entre 75 e 100 %"
)

### Salva o mapa dentro do folder 'mapas_html'

In [ ]:
# Gerar o HTML do mapa como uma string
mapa_html = mapa._repr_html_()

# Salvar a string em um arquivo com encoding UTF-8
caminho_mapa = "mapas_html/mapa_mvp.html"
if not os.path.exists("mapas_html"):
    os.makedirs("mapas_html")
with open(caminho_mapa, "w", encoding="utf-8") as arquivo:
    arquivo.write(mapa_html)

# Exibe o mapa
mapa